<a href="https://colab.research.google.com/github/dstiff-clgx/2019-Hackathon/blob/master/Manage_NLP_Hackathon_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Connecting to Google Cloud storage

The MLS data sets is stored in a Google Cloud storage bucket. To access the dataset, you must first specify your project ID and the bucket name.


In [1]:
project_id = 'clgx-analytics2-65bd'

bucket_name = 'clgx-analytics2-tiger-team'

In order to access Google Cloud storage, we must authenticate. (This only needs to be done once.)


In [ ]:
# !pip install google-colab    # If google-colab is not installed on your VM, uncomment this line

# from google.colab import auth
# auth.authenticate_user()

     |████████████████████████████████| 81kB 4.3MB/s 
     |████████████████████████████████| 71kB 7.3MB/s 
     |████████████████████████████████| 112kB 6.6MB/s 
     |████████████████████████████████| 768kB 7.0MB/s 
     |████████████████████████████████| 8.0MB 9.3MB/s 
     |████████████████████████████████| 10.1MB 22.8MB/s 
     |████████████████████████████████| 61kB 29.1MB/s 
     |████████████████████████████████| 491kB 59.3MB/s 
  Stored in directory: /home/dstiff_solutions_corelogic_com/.cache/pip/wheels/38/0d/59/701e300a337b2a2e07b27fe74dbfff0bc56ac58f711566ee67
  Stored in directory: /home/dstiff_solutions_corelogic_com/.cache/pip/wheels/4a/45/47/1e126be9d4605e71f00d6e6fb151611f2f4cb9770b050c7d2d
  Stored in directory: /home/dstiff_solutions_corelogic_com/.cache/pip/wheels/72/bf/f4/b68fa69596986881b397b18ff2b9af5f8181233aadcc9f76fd
Successfully built google-colab portpicker tornado
ERROR: distributed 1.21.8 requires msgpack, which is not installed.
  Found existing installat

ModuleNotFoundError: ignored

Set the project ID for gcloud (_Is this necessary?_)


In [2]:
!gcloud config set project {project_id}

Updated property [core/project].


# Load Multiple Listing Service (MLS) dataset into Pandas data frame

`pandas` can read a file directly from Google Cloud storage. The MLS data file is quite large, so it can take some time to read it into a data frame.

`pandas` does not always assign the correct data type to columns in a CSV file. But you can set specific column types in `read_csv` by using a `dtype` dictionary. The `pandas` character type is `object`.

Also include any date columns in the `parse_dates` list to automatically convert them into `datetime64` types.

In [4]:
!pip install gcsfs    # If gcsfs is not installed on your VM, uncomment this line
import pandas as pd

mls_df = pd.read_csv('gs://clgx-analytics2-tiger-team/Closed_Listings_06037_SFR_2017_or_Later_Tabular.csv',
                     dtype={'FA_APN':'object',
                            'CMAS_Zip5':'object',
                            'CMAS_FIPS_CODE':'object'},
                     parse_dates=['ListDate','CloseDate'])

_call non-retriable exception: Anonymous caller does not have storage.objects.get access to the Google Cloud Storage object.
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/gcsfs/core.py", line 487, in _call
    validate_response(r, path)
  File "/usr/local/lib/python3.6/dist-packages/gcsfs/core.py", line 130, in validate_response
    raise HttpError(error)
gcsfs.utils.HttpError: Anonymous caller does not have storage.objects.get access to the Google Cloud Storage object.


HttpError: ignored

Each property listing has a unique ID, which is a combination of its parcel number `FA_APN` and its listing date `ListDate`. All of the listings are for single-family properties in Los Angeles county with listing dates on or after January 1, 2017.

In [ ]:
mls_df[['ID','FA_APN','ListDate',
        'CMAS_SIT_HSE_NBR_1_NZ','CMAS_SIT_STR_NAME_1_NZ','CMAS_PROPERTY_CITY_1','CMAS_PROPERTY_STATE_1','CMAS_Zip5']].head()

The listing `ListPrice` and closed `ClosePrice` prices are avaiable for each property, along with the listing date `ListDate` and closing date `CloseDate`. You can calculate each property's "days-on-market" by subtracting the `ListDate` from the `CloseDate`.

There are some erroneous dates in the MLS dataset, so some of the calculated days-on-market may be odd.

In [ ]:
mls_df['DaysOnMarket'] = mls_df['CloseDate'] - mls_df['ListDate']

display(mls_df[['ListDate','CloseDate','DaysOnMarket']].head())

display(mls_df['DaysOnMarket'].describe())

In the MLS database, all of the listings were marked as "for sale" as opposed to "for rent". But there are some errors in this field, so there are some rental properties included in the dataset.


In [ ]:
(mls_df.ClosePrice < 10000).sum()

# Text Information in the MLS Dataset

The dataset includes the most-populated text fields in the MLS database for Los Angeles county. These are the fields that realtors use to describe properties to potential buyers and include in listing sheets and advertisements.

Some of the fields are not necessarily public (e.g., `AgentRemarks`). For the most part, the field names provide a good description of the information included in each field.

Here are the average number of characacters included in each field across all listings:

In [ ]:
text_desc_cols = ['AgentRemarks','Appliances','Cooling',
                  'Directions','GarageStyle','Heating',
                  'LotDesc','ParkingFeatures','Pool',
                  'PublicRemarks','Roof','RoomsDiningDescription',
                  'RoomsLaundryDescription','RoomsOtherDescription','StoriesDesc',
                  'Style','UtilitiesSewer','UtilitiesWater',
                  'ViewDescription','Zoning','Exterior',
                  'Fencing','Floors','SecurityFeatures',
                  'Utilities','HeatingFuel','IrrigationSource',
                  'Amenities1','Amenities2']

mls_df[text_desc_cols].fillna('').astype(str).apply(lambda x:x.str.len()).mean().sort_values(ascending=False)

The data set also contains a field `AllText` that combines all of the text fields into a single string. The Google AutoML NLP models require that all text appears in a single string.

Each field is denoted by its field name followed by a colon. The fields are separated by semi-colons. (Does including the field names make it more difficult for AutoML to fit accurate NLP models?)

In [ ]:
pd.set_option('display.max_colwidth', None)

display(mls_df[text_desc_cols].head(n=1).transpose())

display(mls_df[['ID','AllText']].head(n=1))

# Creating Training Labels for AutoML

Google AutoML can be used to build a NLP model that classifies records based on text information. But before building AutoML, it is necessary to categorize the data set records and create training labels for each category.

The dataset already contains one set of training labels for the listings categrozied by the ratio of ClosePrice to ListPrice. Depending on market conditions, more attractive properties will sell at larger premiums to their listing prices than less attractive properties. We may be able to predict the range of this premium with an NLP model that uses the contents of the `AllText` fields as an input.

The categories/labels are designed as follow:

`IF ClosePrice/ListPrice > 1.10 THEN ClosePriceListPrice_Ratio_Cat = 4` 
 
`IF ClosePrice/ListPrice > 1.05 AND ClosePrice/ListPrice <= 1.10 THEN ClosePriceListPrice_Ratio_Cat = 3`

`IF ClosePrice/ListPrice > 1.00 AND ClosePrice/ListPrice <= 1.05 THEN ClosePriceListPrice_Ratio_Cat = 2`

`IF ClosePrice/ListPrice > 0.95 AND ClosePrice/ListPrice <= 1.00 THEN ClosePriceListPrice_Ratio_Cat = 1`

`ELSE ClosePriceListPrice_Ratio_Cat = 0`  

In [ ]:
mls_df['ClosePriceListPrice_Ratio_Cat'].value_counts(sort=False)

You can also create your own categories (with training labels). Maybe five categories of `ClosePrice` to `ListPrice` ratios is too many. We can create another label that separates the listings into three categories.

In [ ]:
# Function to set label for one record
def func(row):
    if row['ClosePrice']/row['ListPrice'] > 1.025:
        return 'Premium'
    elif (row['ClosePrice']/row['ListPrice'] <= 1.025 and
          row['ClosePrice']/row['ListPrice'] > 0.975):
        return 'Normal'
    else:
        return 'Discount'

# Apply function to all records
mls_df['ClosePriceListPrice_Ratio_Cat2'] = mls_df.apply(func, axis=1)

mls_df['ClosePriceListPrice_Ratio_Cat2'].value_counts(sort=False)